<a href="https://colab.research.google.com/github/viincci/Azora/blob/main/CompactBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q \
bitmex matplotlib numpy pandas python-telegram-bot python-dotenv yfinance pyTelegramBotAPI

In [ ]:
TOKEN = "7544663916:AAHHqwomrrank1Z3v97RQH1Y1Yb7lNo58bo"
CHAT_ID = "6858476757"

API_KEY = "lcVSkBQ1oXGVqmuLd7BAF3DI"
API_SECRET="PrfztqHYbpqz7xLGkyjGdZpgY8u4jJcUW3yXwUI1Ry6RbCxp"

In [ ]:

# -*- coding: utf-8 -*-
import asyncio
import json
import logging
import os
import threading
import time
import telebot
import pytz
from datetime import datetime, timedelta
from queue import Queue

import bitmex
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import pandas as pd
import telegram
from telegram import Bot, Update
from telegram.error import TelegramError
from telegram.ext import Application, CommandHandler, ContextTypes
from dotenv import load_dotenv
import yfinance as yf

In [ ]:

# Load .env file
load_dotenv()


# Set the correct time zone
utc_now = datetime.utcnow()
sast = pytz.timezone('Africa/Johannesburg')
sast_now = utc_now.replace(tzinfo=pytz.utc).astimezone(sast)

In [ ]:

class TelegramLogger:
    _instance = None

    def __new__(cls, message=None):
        """
        Singleton implementation with optional message sending.

        :param message: Optional message to send
        """
        if not cls._instance:
            cls._instance = super(TelegramLogger, cls).__new__(cls)
            cls._instance._initialize()

        # Send message if provided
        if message:
            cls._instance.send_message(message)

        return cls._instance

    def _initialize(self):
        """
        Initialize the Telegram Logger singleton.
        """
        # Load environment variables
        load_dotenv()

        # Initialize bot
        try:
            self.bot = telebot.TeleBot(TOKEN)
            self.chat_id = os.getenv(CHAT_ID)
        except Exception as e:
            print(f"Failed to initialize Telegram bot: {e}")
            self.bot = None
            self.chat_id = None

    def send_message(self, message):
        """
        Send a message to Telegram.

        :param message: Message to send
        """
        # Skip sending if bot is not initialized
        if not self.bot or not self.chat_id:
            return

        try:
            # Send message
            self.bot.send_message(self.chat_id, message)
        except Exception as e:
            # Silently handle errors to prevent logging disruption
            print(f"Error sending Telegram message: {e}")

def configure_logging():
    """
    Configure logging with a custom Telegram handler.

    :return: Configured logger
    """
    # Create a custom logging handler
    class CustomLoggingHandler(logging.Handler):
        def emit(self, record):
            try:
                # Format the log message
                log_message = self.format(record)

                # Send to Telegram using TelegramLogger
                TelegramLogger(log_message)
            except Exception as e:
                print(f"Error in custom logging handler: {e}")

    # Configure the logger
    logger = logging.getLogger(__name__)
    logger.setLevel(logging.INFO)

    # Create the custom handler
    custom_handler = CustomLoggingHandler()
    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
    custom_handler.setFormatter(formatter)

    # Clear existing handlers to prevent duplicate logging
    logger.handlers.clear()

    # Add the custom handler to the logger
    logger.addHandler(custom_handler)

    # Add standard StreamHandler
    stream_handler = logging.StreamHandler()
    stream_handler.setFormatter(formatter)
    logger.addHandler(stream_handler)

    return logger

In [ ]:
logger = configure_logging()
# Set the correct time zone
def get_sast_time():
    utc_now = datetime.utcnow()
    sast = pytz.timezone('Africa/Johannesburg')
    return utc_now.replace(tzinfo=pytz.utc).astimezone(sast)

In [ ]:

class BitMEXTestAPI:
    def __init__(self, api_key, api_secret, test=True, symbol='SOL-USD'):
        """
        Initialize BitMEX API client

        :param api_key: BitMEX API key
        :param api_secret: BitMEX API secret
        :param test: Whether to use testnet (default True)
        :param symbol: Trading symbol (default SOL-USD)
        """
        try:
            self.client = bitmex.bitmex(
                test=test,
                api_key=api_key,
                api_secret=api_secret
            )
            self.symbol = symbol

            # Log initialization
            network_type = 'testnet' if test else 'mainnet'
            logger.info(f"BitMEXTestAPI initialized for {symbol} on {network_type}")
            print(f"BitMEXTestAPI initialized for {symbol} on {network_type}")

        except Exception as e:
            logger.error(f"Initialization error: {str(e)}")
            print(f"Initialization error: {str(e)}")
            raise

    def get_profile_info(self):
        """
        Retrieve comprehensive account profile information

        :return: Dictionary with user, balance, and position details
        """
        try:
            # Get user information
            user_info = self.client.User.User_get().result()[0]

            # Get account balance
            margin = self.client.User.User_getMargin().result()[0]

            # Get open positions
            positions = self.client.Position.Position_get(
                filter=json.dumps({"symbol": self.symbol})
            ).result()[0]

            # Format profile information
            profile_info = {
                "user": {
                    "id": user_info.get('id'),
                    "username": user_info.get('username'),
                    "email": user_info.get('email'),
                    "account": user_info.get('account')
                },
                "balance": {
                    "wallet_balance": margin.get('walletBalance'),
                    "margin_balance": margin.get('marginBalance'),
                    "available_margin": margin.get('availableMargin'),
                    "unrealized_pnl": margin.get('unrealisedPnl'),
                    "realized_pnl": margin.get('realisedPnl')
                },
                "positions": [{
                    "symbol": pos.get('symbol'),
                    "current_qty": pos.get('currentQty'),
                    "avg_entry_price": pos.get('avgEntryPrice'),
                    "leverage": pos.get('leverage'),
                    "liquidation_price": pos.get('liquidationPrice'),
                    "unrealized_pnl": pos.get('unrealisedPnl'),
                    "realized_pnl": pos.get('realisedPnl')
                } for pos in positions] if positions else []
            }

            # Logging profile details
            logger.info("Profile information retrieved successfully")
            print("Profile information retrieved successfully")

            logger.info(f"Account: {profile_info['user']['username']}")
            print(f"Account: {profile_info['user']['username']}")

            logger.info(f"Wallet Balance: {profile_info['balance']['wallet_balance'] / 100000000:.8f} XBT")
            print(f"Wallet Balance: {profile_info['balance']['wallet_balance'] / 100000000:.8f} XBT")

            logger.info(f"Available Margin: {profile_info['balance']['available_margin'] / 100000000:.8f} XBT")
            print(f"Available Margin: {profile_info['balance']['available_margin'] / 100000000:.8f} XBT")

            if profile_info['positions']:
                for pos in profile_info['positions']:
                    logger.info(f"Position: {pos['symbol']} | Qty: {pos['current_qty']} | Entry: {pos['avg_entry_price']}")
                    print(f"Position: {pos['symbol']} | Qty: {pos['current_qty']} | Entry: {pos['avg_entry_price']}")
            else:
                logger.info("No open positions")
                print("No open positions")

            return profile_info

        except Exception as e:
            logger.error(f"Error getting profile information: {str(e)}")
            print(f"Error getting profile information: {str(e)}")
            return None

    def get_candle(self, timeframe='1m', count=100):
        """
        Retrieve candlestick (OHLCV) data for the specified symbol

        :param timeframe: Candle timeframe (default '1m' for 1 minute)
        :param count: Number of candles to retrieve (default 100)
        :return: List of candle dictionaries or None if error occurs
        """
        try:
            # Mapping of timeframe to BitMEX binSize parameter
            timeframe_map = {
                '1m': '1',
                '5m': '5',
                '1h': '1h',
                '1d': '1d'
            }

            # Validate and get the correct bin size
            bin_size = timeframe_map.get(timeframe)
            if not bin_size:
                raise ValueError(f"Invalid timeframe. Supported: {', '.join(timeframe_map.keys())}")

            # Retrieve candle data
            candles = self.client.Trade.Trade_getBucketed(
                symbol=self.symbol,
                binSize=bin_size,
                count=count,
                reverse=True  # Most recent candles first
            ).result()[0]

            # Format candle data
            formatted_candles = [{
                'timestamp': candle['timestamp'],
                'open': candle['open'],
                'high': candle['high'],
                'low': candle['low'],
                'close': candle['close'],
                'volume': candle['volume']
            } for candle in candles]

            # Log retrieval success
            logger.info(f"Retrieved {len(formatted_candles)} {timeframe} candles for {self.symbol}")
            print(f"Retrieved {len(formatted_candles)} {timeframe} candles for {self.symbol}")

            return formatted_candles

        except Exception as e:
            logger.error(f"Error retrieving candle data: {str(e)}")
            print(f"Error retrieving candle data: {str(e)}")
            return None

    def open_test_position(self, side="Buy", quantity=100, order_type="Market"):
        """
        Open a test trading position

        :param side: Buy or Sell
        :param quantity: Number of contracts
        :param order_type: Type of order (default Market)
        :return: Order details or None if error
        """
        try:
            logger.info(f"Opening test {side} position for {quantity} contracts")

            # Execute the order
            order = self.client.Order.Order_new(
                symbol=self.symbol,
                side=side,
                orderQty=quantity,
                ordType=order_type
            ).result()[0]

            # Log order details
            logger.info(f"Order placed: {order['ordStatus']} | OrderID: {order['orderID']}")
            print(f"Order placed: {order['ordStatus']} | OrderID: {order['orderID']}")

            logger.info(f"Order details: {side} {quantity} contracts at {order.get('price', 'market price')}")
            print(f"Order details: {side} {quantity} contracts at {order.get('price', 'market price')}")

            # Wait for order to settle
            time.sleep(2)
            self.get_profile_info()

            return order

        except Exception as e:
            logger.error(f"Error opening test position: {str(e)}")
            print(f"Error opening test position: {str(e)}")
            return None

    def close_all_positions(self):
        """
        Close all open positions for the current symbol

        :return: True if successful, None if error
        """
        try:
            # Get current positions
            positions = self.client.Position.Position_get(
                filter=json.dumps({"symbol": self.symbol})
            ).result()[0]

            if not positions:
                logger.info("No positions to close")
                print("No positions to close")
                return None

            # Close each position
            for position in positions:
                symbol = position['symbol']
                current_qty = position['currentQty']

                if current_qty == 0:
                    logger.info(f"No open position for {symbol}")
                    print(f"No open position for {symbol}")
                    continue

                # Determine closing side
                side = "Sell" if current_qty > 0 else "Buy"
                qty = abs(current_qty)

                logger.info(f"Closing position: {symbol} | {current_qty} contracts | Side: {side} | Qty: {qty}")
                print(f"Closing position: {symbol} | {current_qty} contracts | Side: {side} | Qty: {qty}")

                # Place closing order
                order = self.client.Order.Order_new(
                    symbol=symbol,
                    side=side,
                    orderQty=qty,
                    ordType="Market"
                ).result()[0]

                logger.info(f"Position closed: {order['ordStatus']} | OrderID: {order['orderID']}")
                print(f"Position closed: {order['ordStatus']} | OrderID: {order['orderID']}")

            # Wait for orders to settle
            time.sleep(2)
            self.get_profile_info()

            return True

        except Exception as e:
            logger.error(f"Error closing positions: {str(e)}")
            print(f"Error closing positions: {str(e)}")
            return None

    def run_test_sequence(self):
        """
        Run a comprehensive test sequence of trading operations

        :return: Final profile information or None if error
        """
        try:
            logger.info("Starting test sequence")
            print("Starting test sequence")

            # Initial profile
            logger.info("=== INITIAL PROFILE ===")
            print("=== INITIAL PROFILE ===")
            self.get_profile_info()

            # Open long position
            logger.info("=== OPENING LONG POSITION ===")
            print("=== OPENING LONG POSITION ===")
            self.open_test_position(side="Buy", quantity=1)

            # Wait and check profile
            wait_time = 1
            logger.info(f"Waiting for {wait_time} seconds...")
            print(f"Waiting for {wait_time} seconds...")
            time.sleep(wait_time)
            self.get_profile_info()

            # Open short position
            logger.info("=== OPENING SHORT POSITION ===")
            print("=== OPENING SHORT POSITION ===")
            self.open_test_position(side="Sell", quantity=1)

            # Wait and check profile
            logger.info(f"Waiting for {wait_time} seconds...")
            time.sleep(wait_time)
            self.get_profile_info()

            # Close all positions
            logger.info("=== CLOSING ALL POSITIONS ===")
            print("=== CLOSING ALL POSITIONS ===")
            self.close_all_positions()

            # Final profile check
            logger.info("=== FINAL PROFILE ===")
            print("=== FINAL PROFILE ===")
            final_profile = self.get_profile_info()

            logger.info("Test sequence completed successfully")
            print("Test sequence completed successfully")
            return final_profile

        except Exception as e:
            logger.error(f"Error in test sequence: {str(e)}")
            print(f"Error in test sequence: {str(e)}")
            return None

In [ ]:


class SMC:
    def __init__(self, api_key="", api_secret="", test=True, symbol="SOL-USD",
                 timeframe="15m", risk_per_trade=0.02, lookback_periods=100):
        """
        Initialize the SMC trading class
        """
        self.api_key = api_key
        self.api_secret = api_secret
        self.test = test
        self.symbol = symbol
        self.timeframe = timeframe
        self.risk_per_trade = risk_per_trade
        self.lookback_periods = lookback_periods

        # Initialize BitMEX API client
        self.api = BitMEXTestAPI(
            api_key=self.api_key,
            api_secret=self.api_secret,
            test=self.test
        )

        # Trading state
        self.initial_balance = 0
        self.current_balance = 0
        self.df = pd.DataFrame()
        self.in_trade = False
        self.entry_price = 0
        self.stop_loss = 0
        self.take_profit = 0
        self.position_size = 0
        self.trade_type = None
        self.trades = []
        self.equity_curve = []

        logger.info(f"BitMEXLiveTrader initialized for {symbol} on {timeframe} timeframe")
    def get_market_data(self):
        """
        Fetch market data from BitMEX API or fallback to yfinance
        """
        try:
            print(f"Fetching {self.symbol} market data from BitMEX")
            data = self.api.get_candle(
                symbol=self.symbol,
                timeframe=self.timeframe
            )
            df = pd.DataFrame(data)
            print(f"Retrieved {len(df)} candles from BitMEX")
            self.df = df
            # Normalize column names to lowercase
            self.df.columns = [col.lower() for col in self.df.columns]
            print(f"BitMEX columns: {self.df.columns.tolist()}")
            # Add columns for SMC analysis
            self.df['higher_high'] = False
            self.df['lower_low'] = False
            self.df['bos_up'] = False
            self.df['bos_down'] = False
            self.df['choch_up'] = False
            self.df['choch_down'] = False
            self.df['bullish_fvg'] = False
            self.df['bearish_fvg'] = False
            return df

        except Exception as e:
            print(f"Failed to get data from BitMEX API: {str(e)}. Falling back to yfinance.")

            # Fix ticker symbol for yfinance
            crypto_ticker = self.symbol.replace('USD', '-USD')  # Ensure SOLUSD format
            if not crypto_ticker.endswith('USD'):
                crypto_ticker += 'USD'  # Ensure it ends with USD

            sast_now = get_sast_time()
            end_date = sast_now
            start_date = end_date - timedelta(days=7)

            data = yf.download(
                crypto_ticker,
                start=start_date.strftime('%Y-%m-%d'),
                end=end_date.strftime('%Y-%m-%d'),
                interval=self.timeframe
            )

            print(f"Retrieved {len(data)} candles from yfinance")
            self.df = data
            if not data.empty:
                # Handle potential MultiIndex from yfinance
                if isinstance(self.df.columns, pd.MultiIndex):
                    print(f"MultiIndex detected: {self.df.columns.tolist()}")
                    # Flatten MultiIndex to just the second level (e.g., 'open', 'high', etc.)
                    self.df.columns = [col[0].lower() if isinstance(col, tuple) else col.lower()
                                     for col in self.df.columns]
                else:
                    # Simple Index, just convert to lowercase
                    self.df.columns = [col.lower() for col in self.df.columns]

                print(f"Normalized yfinance columns: {self.df.columns.tolist()}")

                # Verify required columns exist
                required_columns = ['open', 'high', 'low', 'close']
                missing_columns = [col for col in required_columns if col not in self.df.columns]
                if missing_columns:
                    print(f"Warning: Missing required columns: {missing_columns}")
                    return self.df  # Return early if critical columns are missing

                # Add columns for SMC analysis
                self.df['higher_high'] = False
                self.df['lower_low'] = False
                self.df['bos_up'] = False
                self.df['bos_down'] = False
                self.df['choch_up'] = False
                self.df['choch_down'] = False
                self.df['bullish_fvg'] = False
                self.df['bearish_fvg'] = False
            else:
                print("No data retrieved from yfinance")
            return data
    def identify_structure(self):
        """Identify market structure including highs, lows, BOS and CHoCH"""
        logger.info("Identifying market structure")
        print("Identifying market structure")
        df = self.df

        # Identify Higher Highs and Lower Lows (using a 5-candle lookback)
        window = 5
        for i in range(window, len(df)):
            # Higher High
            if df.iloc[i]['high'] > max(df.iloc[i-window:i]['high']):
                df.loc[df.index[i], 'higher_high'] = True

            # Lower Low
            if df.iloc[i]['low'] < min(df.iloc[i-window:i]['low']):
                df.loc[df.index[i], 'lower_low'] = True

        # Identify Break of Structure (BOS)
        prev_structure_high = df.iloc[0]['high']
        prev_structure_low = df.iloc[0]['low']
        structure_points_high = []
        structure_points_low = []

        for i in range(1, len(df)):
            current_high = df.iloc[i]['high']
            current_low = df.iloc[i]['low']

            # Track significant structure points
            if df.iloc[i]['higher_high']:
                structure_points_high.append((i, current_high))

            if df.iloc[i]['lower_low']:
                structure_points_low.append((i, current_low))

            # BOS Up: Price breaks above recent structure high
            if len(structure_points_high) >= 2:
                last_high_idx, last_high = structure_points_high[-1]
                prev_high_idx, prev_high = structure_points_high[-2]

                if current_low > prev_high and i > last_high_idx + 1:
                    df.loc[df.index[i], 'bos_up'] = True
                    logger.info(f"Bullish BOS detected at index {i}, price: {current_low}")
                    print(f"Bullish BOS detected at index {i}, price: {current_low}")

            # BOS Down: Price breaks below recent structure low
            if len(structure_points_low) >= 2:
                last_low_idx, last_low = structure_points_low[-1]
                prev_low_idx, prev_low = structure_points_low[-2]

                if current_high < prev_low and i > last_low_idx + 1:
                    df.loc[df.index[i], 'bos_down'] = True
                    logger.info(f"Bearish BOS detected at index {i}, price: {current_high}")
                    print(f"Bearish BOS detected at index {i}, price: {current_high}")

        # Identify Change of Character (CHoCH)
        for i in range(window+1, len(df)):
            # Bullish CHoCH: After BOS up, creates higher low
            if df.iloc[i-1]['bos_up']:
                recent_lows = df.iloc[i-window:i]['low'].tolist()
                if min(recent_lows[:-1]) < recent_lows[-1]:
                    df.loc[df.index[i], 'choch_up'] = True
                    logger.info(f"Bullish CHoCH detected at index {i}")
                    print(f"Bullish CHoCH detected at index {i}")

            # Bearish CHoCH: After BOS down, creates lower high
            if df.iloc[i-1]['bos_down']:
                recent_highs = df.iloc[i-window:i]['high'].tolist()
                if max(recent_highs[:-1]) > recent_highs[-1]:
                    df.loc[df.index[i], 'choch_down'] = True
                    logger.info(f"Bearish CHoCH detected at index {i}")
                    print(f"Bearish CHoCH detected at index {i}")

        return df

    def identify_fvg(self):
        """Identify Fair Value Gaps (FVGs)"""
        logger.info("Identifying Fair Value Gaps")
        print("Identifying Fair Value Gaps")
        df = self.df

        # Create separate columns for FVG values to avoid tuple storage issues
        if 'bullish_fvg_low' not in df.columns:
            df['bullish_fvg_low'] = np.nan
            df['bullish_fvg_high'] = np.nan
            df['bullish_fvg_sl_index'] = np.nan

        if 'bearish_fvg_low' not in df.columns:
            df['bearish_fvg_low'] = np.nan
            df['bearish_fvg_high'] = np.nan
            df['bearish_fvg_sl_index'] = np.nan

        # Add mitigation tracking columns
        if 'bullish_fvg_mitigated' not in df.columns:
            df['bullish_fvg_mitigated'] = False

        if 'bearish_fvg_mitigated' not in df.columns:
            df['bearish_fvg_mitigated'] = False

        for i in range(2, len(df)):
            # Bullish FVG: Previous candle's low > Current candle's high
            if df.iloc[i-2]['low'] > df.iloc[i]['high']:
                # Check Gann Box (0-0.5 range for bullish FVGs)
                high_point = df.iloc[i-2]['high']
                low_point = df.iloc[i]['low']
                price_range = high_point - low_point
                fvg_low = df.iloc[i]['high']
                fvg_high = df.iloc[i-2]['low']

                # Calculate relative position in Gann Box (0 to 1)
                if price_range > 0:
                    relative_pos = (fvg_high - low_point) / price_range

                    # Only mark valid FVGs within 0-0.5 Gann range
                    if 0 <= relative_pos <= 0.5:
                        df.loc[df.index[i], 'bullish_fvg'] = True
                        # Store FVG values in separate columns
                        df.loc[df.index[i], 'bullish_fvg_low'] = fvg_low
                        df.loc[df.index[i], 'bullish_fvg_high'] = fvg_high
                        df.loc[df.index[i], 'bullish_fvg_sl_index'] = i

                        logger.info(f"Bullish FVG detected at index {i}, range: {fvg_low}-{fvg_high}")
                        print(f"Bullish FVG detected at index {i}, range: {fvg_low}-{fvg_high}")

            # Bearish FVG: Previous candle's high < Current candle's low
            if df.iloc[i-2]['high'] < df.iloc[i]['low']:
                # Check Gann Box (0.5-1 range for bearish FVGs)
                high_point = df.iloc[i]['high']
                low_point = df.iloc[i-2]['low']
                price_range = high_point - low_point
                fvg_low = df.iloc[i-2]['high']
                fvg_high = df.iloc[i]['low']

                # Calculate relative position in Gann Box (0 to 1)
                if price_range > 0:
                    relative_pos = (fvg_high - low_point) / price_range

                    # Only mark valid FVGs within 0.5-1 Gann range
                    if 0.5 <= relative_pos <= 1:
                        df.loc[df.index[i], 'bearish_fvg'] = True
                        # Store FVG values in separate columns
                        df.loc[df.index[i], 'bearish_fvg_low'] = fvg_low
                        df.loc[df.index[i], 'bearish_fvg_high'] = fvg_high
                        df.loc[df.index[i], 'bearish_fvg_sl_index'] = i

                        logger.info(f"Bearish FVG detected at index {i}, range: {fvg_low}-{fvg_high}")
                        print(f"Bearish FVG detected at index {i}, range: {fvg_low}-{fvg_high}")

        return df

    def check_fvg_mitigation(self, current_idx):
        """Check if any previously identified FVGs have been mitigated"""
        df = self.df

        # Loop through all previous candles
        for i in range(current_idx):
            # Check if the candle had a bullish FVG
            if df.iloc[i].get('bullish_fvg', False) and pd.notna(df.iloc[i].get('bullish_fvg_low')):
                fvg_low = df.iloc[i]['bullish_fvg_low']
                fvg_high = df.iloc[i]['bullish_fvg_high']

                # Check if price revisited the FVG area
                for j in range(i+1, current_idx+1):
                    if df.iloc[j]['low'] <= fvg_high and df.iloc[j]['high'] >= fvg_low:
                        # FVG has been mitigated
                        df.loc[df.index[i], 'bullish_fvg_mitigated'] = True
                        logger.info(f"Bullish FVG at index {i} has been mitigated at index {j}")
                        print(f"Bullish FVG at index {i} has been mitigated at index {j}")
                        break

            # Check if the candle had a bearish FVG
            if df.iloc[i].get('bearish_fvg', False) and pd.notna(df.iloc[i].get('bearish_fvg_low')):
                fvg_low = df.iloc[i]['bearish_fvg_low']
                fvg_high = df.iloc[i]['bearish_fvg_high']

                # Check if price revisited the FVG area
                for j in range(i+1, current_idx+1):
                    if df.iloc[j]['low'] <= fvg_high and df.iloc[j]['high'] >= fvg_low:
                        # FVG has been mitigated
                        df.loc[df.index[i], 'bearish_fvg_mitigated'] = True
                        logger.info(f"Bearish FVG at index {i} has been mitigated at index {j}")
                        print(f"Bearish FVG at index {i} has been mitigated at index {j}")
                        break

        return df

    def execute_trades(self):
        """Execute trades based on SMC signals"""
        logger.info("Starting trade execution backtesting")
        print("Starting trade execution backtesting")
        df = self.df

        # Iterate through each candle for backtesting
        for i in range(5, len(df)):
            current_price = df.iloc[i]['close']

            # Check if we're in a trade
            if self.in_trade:
                # Check if stop loss hit
                if (self.trade_type == 'long' and df.iloc[i]['low'] <= self.stop_loss) or \
                   (self.trade_type == 'short' and df.iloc[i]['high'] >= self.stop_loss):
                    #

                    # Stop loss hit
                     self.execute_signal({
                     'price': self.take_profit,  # Use the take_profit price as the exit price
                     'take_profit': self.take_profit,
                     'action': "exit",
                     'reason': 'takeprofit',
                       })

                     #self.execute_signal(signal)
                     logger.info("Stop loss hit ❎❎❎")


                # Check if take profit hit
                elif (self.trade_type == 'long' and df.iloc[i]['high'] >= self.take_profit) or \
                     (self.trade_type == 'short' and df.iloc[i]['low'] <= self.take_profit):
                    # Take profit hit
                    self.execute_signal({
                               'take_profit': self.take_profit,
                               'price': self.take_profit,
                                'action': "exit",
                                'reason': 'takeprofit',
                               })
                    #self.execute_signal(signal)
                    logger.info("Take profit hit 💲✔️✔️")


            else:
                # Update FVG mitigation status
                self.check_fvg_mitigation(i)

                # Check for new trade setups

                # Bullish setup: BOS up + CHoCH up + unmitigated bullish FVG
                if df.iloc[i-1]['bos_up'] and df.iloc[i]['choch_up']:
                    # Look back for unmitigated bullish FVGs
                    for j in range(i-10, i):
                        if j >= 0 and df.iloc[j].get('bullish_fvg', False) and not df.iloc[j].get('bullish_fvg_mitigated', False):
                            fvg_low = df.iloc[j]['bullish_fvg_low']
                            fvg_high = df.iloc[j]['bullish_fvg_high']
                            sl_idx = int(df.iloc[j]['bullish_fvg_sl_index'])

                            # Check if price is near the FVG
                            if fvg_low <= current_price <= fvg_high:
                                # Setup stop loss at the low of the FVG-forming candle
                                stop_loss = df.iloc[sl_idx]['low']

                                # Find recent structure low for take profit
                                recent_lows = df.iloc[i-20:i]['low'].tolist()
                                min_idx = recent_lows.index(min(recent_lows))
                                take_profit = df.iloc[i-20+min_idx]['low']

                                stop_loss = df.iloc[sl_idx]['low']
                                risk = current_price - stop_loss
                                take_profit = current_price + (risk * 2)  # 2:1 risk-reward
                                signal = {
                                    'action': 'entry',
                                    'side': 'long',
                                    'price': current_price,
                                    'stop_loss': stop_loss,
                                    'take_profit': take_profit
                                }
                                # Enter long trade
                                self.execute_signal(signal)
                                logger.info("Enter long🔵(Buy) 📈📈📈 trade 🐃🐃🐃")

                                break

                # Bearish setup: BOS down + CHoCH down + unmitigated bearish FVG
                if df.iloc[i-1]['bos_down'] and df.iloc[i]['choch_down']:
                    # Look back for unmitigated bearish FVGs
                    for j in range(i-10, i):
                        if j >= 0 and df.iloc[j].get('bearish_fvg', False) and not df.iloc[j].get('bearish_fvg_mitigated', False):
                            fvg_low = df.iloc[j]['bearish_fvg_low']
                            fvg_high = df.iloc[j]['bearish_fvg_high']
                            sl_idx = int(df.iloc[j]['bearish_fvg_sl_index'])

                            # Check if price is near the FVG
                            if fvg_low <= current_price <= fvg_high:
                                # Setup stop loss at the high of the FVG-forming candle
                                stop_loss = df.iloc[sl_idx]['high']

                                # Find recent structure high for take profit
                                recent_highs = df.iloc[i-20:i]['high'].tolist()
                                max_idx = recent_highs.index(max(recent_highs))
                                take_profit = df.iloc[i-20+max_idx]['high']

                                # Enter short trade
                                stop_loss = df.iloc[sl_idx]['high']
                                risk = stop_loss - current_price
                                take_profit = current_price - (risk * 2)  # 2:1 risk-reward

                                signal = {
                                    'action': 'entry',
                                    'side': 'short',
                                    'price': current_price,
                                    'stop_loss': stop_loss,
                                    'take_profit': take_profit
                                }
                                self.execute_signal(signal)
                                logger.info("Enter short🔴(Sell)📉📉📉 trade 🐻🐻🐻")
                                break

        # Close any open trade at the end of testing
        #if self.in_trade:
            # Take profit hit
             #signal = {
                              # 'price': self.df.iloc[-1]['close'],
                               # 'action': "exit",
                                #'reason': 'stoploss',
                               #}
            #self.execute_signal(signal)
            #logger.info("Close any open trade at the end of testing")
            #self.exit_trade(len(df) - 1, df.iloc[-1]['close'], 'end_of_test')

        logger.info(f"Trade execution completed with {len(self.trades)} trades")
        print(f"Trade execution completed with {len(self.trades)} trades")

        return self.trades, self.equity_curve





    def calculate_performance(self):
        """Calculate and return performance metrics"""
        if not self.trades:
            logger.warning("No trades to calculate performance metrics")
            print("No trades to calculate performance metrics")
            return {
                'total_trades': 0,
                'win_rate': 0,
                'profit_factor': 0,
                'total_return_pct': 0,
                'max_drawdown_pct': 0
            }

        # Calculate win rate
        winning_trades = [t for t in self.trades if t['pnl'] > 0]
        win_rate = len(winning_trades) / len(self.trades)

        # Calculate profit factor
        gross_profit = sum([t['pnl'] for t in self.trades if t['pnl'] > 0])
        gross_loss = abs(sum([t['pnl'] for t in self.trades if t['pnl'] <= 0]))
        profit_factor = gross_profit / gross_loss if gross_loss > 0 else float('inf')

        # Calculate total return
        total_return = (self.balance - self.initial_balance) / self.initial_balance

        # Calculate maximum drawdown
        peak = self.initial_balance
        max_drawdown = 0

        for balance in self.equity_curve:
            if balance > peak:
                peak = balance
            drawdown = (peak - balance) / peak
            max_drawdown = max(max_drawdown, drawdown)

        performance = {
            'total_trades': len(self.trades),
            'winning_trades': len(winning_trades),
            'losing_trades': len(self.trades) - len(winning_trades),
            'win_rate': win_rate,
            'profit_factor': profit_factor,
            'total_return': self.balance - self.initial_balance,
            'total_return_pct': total_return * 100,
            'max_drawdown_pct': max_drawdown * 100,
            'final_balance': self.balance
        }

        logger.info(f"Performance metrics: Win rate: {win_rate:.2%}, Profit factor: {profit_factor:.2f}, " +
                     f"Return: {total_return:.2%}, Max drawdown: {max_drawdown:.2%}")
        print(f"Performance metrics: Win rate: {win_rate:.2%}, Profit factor: {profit_factor:.2f}, " +
              f"Return: {total_return:.2%}, Max drawdown: {max_drawdown:.2%}")

        return performance

    def visualize_results(self, start_idx=0, end_idx=None):
        """Visualize backtesting results with trades and SMC patterns"""
        if end_idx is None:
            end_idx = len(self.df)

        # Create figure with subplots
        fig, ax = plt.subplots(figsize=(15, 8))

        # Plot price data
        subset = self.df.iloc[start_idx:end_idx]
        ax.plot(subset.index, subset['close'], label='Close Price', color='black', linewidth=1)

        # Plot FVGs
        for i in range(start_idx, min(end_idx, len(self.df))):
            if self.df.iloc[i].get('bullish_fvg', False) and pd.notna(self.df.iloc[i].get('bullish_fvg_low')):
                fvg_low = self.df.iloc[i]['bullish_fvg_low']
                fvg_high = self.df.iloc[i]['bullish_fvg_high']
                mitigated = self.df.iloc[i].get('bullish_fvg_mitigated', False)
                color = 'lightgreen' if not mitigated else 'darkgreen'
                rect = patches.Rectangle((i-0.5, fvg_low), 1, fvg_high-fvg_low, linewidth=1,
                                        edgecolor=color, facecolor=color, alpha=0.3)
                ax.add_patch(rect)

            if self.df.iloc[i].get('bearish_fvg', False) and pd.notna(self.df.iloc[i].get('bearish_fvg_low')):
                fvg_low = self.df.iloc[i]['bearish_fvg_low']
                fvg_high = self.df.iloc[i]['bearish_fvg_high']
                mitigated = self.df.iloc[i].get('bearish_fvg_mitigated', False)
                color = 'lightcoral' if not mitigated else 'darkred'
                rect = patches.Rectangle((i-0.5, fvg_low), 1, fvg_high-fvg_low, linewidth=1,
                                        edgecolor=color, facecolor=color, alpha=0.3)
                ax.add_patch(rect)

        # Plot BOS and CHoCH
        bos_up_idx = subset[subset['bos_up'] == True].index
        bos_down_idx = subset[subset['bos_down'] == True].index
        choch_up_idx = subset[subset['choch_up'] == True].index
        choch_down_idx = subset[subset['choch_down'] == True].index

        ax.scatter(bos_up_idx, subset.loc[bos_up_idx, 'low'], color='green', marker='^', s=100, label='BOS Up')
        ax.scatter(bos_down_idx, subset.loc[bos_down_idx, 'high'], color='red', marker='v', s=100, label='BOS Down')
        ax.scatter(choch_up_idx, subset.loc[choch_up_idx, 'low'], color='blue', marker='^', s=80, label='CHoCH Up')
        ax.scatter(choch_down_idx, subset.loc[choch_down_idx, 'high'], color='purple', marker='v', s=80, label='CHoCH Down')

        # Plot trades
        for trade in self.trades:
            if start_idx <= trade['entry_index'] < end_idx:
                # Entry point
                color = 'green' if trade['type'] == 'long' else 'red'
                marker = '^' if trade['type'] == 'long' else 'v'
                ax.scatter(trade['entry_index'], trade['entry_price'], color=color, marker=marker, s=120, zorder=5)

                # Exit point
                if trade['exit_index'] < end_idx:
                    color = 'green' if trade['pnl'] > 0 else 'red'
                    ax.scatter(trade['exit_index'], trade['exit_price'], color=color, marker='o', s=120, zorder=5)

                    # Connect entry and exit
                    ax.plot([trade['entry_index'], trade['exit_index']],
                           [trade['entry_price'], trade['exit_price']],
                           color=color, linewidth=1, linestyle='--')

                    # Annotate PnL
                    ax.annotate(f"{trade['pnl']:.2f}",
                              (trade['exit_index'], trade['exit_price']),
                              textcoords="offset points",
                              xytext=(0,10),
                              ha='center')

        ax.set_title('SMC Backtest Results')
        ax.legend(loc='best')
        ax.grid(True, alpha=0.3)

        # Plot equity curve
        fig2, ax2 = plt.subplots(figsize=(15, 5))
        ax2.plot(self.equity_curve, label='Account Balance', color='blue')
        ax2.set_title('Equity Curve')
        ax2.grid(True, alpha=0.3)
        ax2.legend()

        plt.tight_layout()
        return fig, fig2

    def execute_signal(self, signal):
        if signal is None:
            logger.info("No trading signal detected")
            print("No trading signal detected")
            return

        if signal['action'] == 'entry':
            self.execute_entry(signal)
        elif signal['action'] == 'exit':
            self.execute_exit(signal)

    def execute_entry(self, signal):
        side = signal['side']
        price = signal['price']
        stop_loss = signal['stop_loss']
        take_profit = signal['take_profit']

        # Get current time for trade recording
        sast_now = get_sast_time()

        # ... (rest of your execute_entry method remains mostly the same)
        try:
            # Execute order via BitMEX API
            order_side = "Buy" if side == "long" else "Sell"
            order_result = self.api.open_test_position(side=order_side, quantity=self.position_size)

            # Update trading state
            self.in_trade = True
            self.trade_type = side
            self.entry_price = price
            self.stop_loss = stop_loss
            self.take_profit = take_profit

            # Record trade
            trade = {
                'entry_time': sast_now.strftime('%Y-%m-%d %H:%M:%S'),
                'entry_price': price,
                'entry_index': len(self.df) - 1,
                'side': side,
                'position_size': self.position_size,
                'stop_loss': stop_loss,
                'take_profit': take_profit,
                'risk_amount': self.risk_per_trade * self.current_balance
            }
            self.trades.append(trade)
            self.equity_curve.append(self.current_balance)

        except Exception as e:
            logger.error(f"Error opening {side} position: {str(e)}")
            print(f"Error opening {side} position: {str(e)}")

    def execute_exit(self, signal):
        reason = signal['reason']
        price = signal['price']
        sast_now = get_sast_time()

        try:
            # Close position via BitMEX API
            self.api.close_all_positions()

            # Calculate profit/loss
            if self.trade_type == 'long':
                pnl = (price - self.entry_price) * self.position_size
            else:  # short
                pnl = (self.entry_price - price) * self.position_size

            # Update the latest trade record
            current_trade = self.trades[-1]
            current_trade.update({
                'exit_time': sast_now.strftime('%Y-%m-%d %H:%M:%S'),
                'exit_price': price,
                'exit_index': len(self.df) - 1,
                'exit_reason': reason,
                'pnl': pnl
            })

            # Update balance
            self.current_balance += pnl
            self.equity_curve.append(self.current_balance)

            # Reset trading state
            self.in_trade = False
            self.trade_type = None
            self.entry_price = 0
            self.stop_loss = 0
            self.take_profit = 0
            self.position_size = 0

        except Exception as e:
            logger.error(f"Error closing position: {str(e)}")
            print(f"Error closing position: {str(e)}")


    def run(self, scan_interval=60):
        sast_now = get_sast_time()
        logger.info(f"Starting BitMEXLiveTrader at {sast_now.strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"Starting BitMEXLiveTrader at {sast_now.strftime('%Y-%m-%d %H:%M:%S')}")

        # Initialize balance
        profile = self.api.get_profile_info()
        self.initial_balance = profile['balance']
        self.current_balance = self.initial_balance
        self.equity_curve = [self.initial_balance]

        try:
            while True:
                sast_now = get_sast_time()
                # Your existing run logic...
                self.get_market_data()
                self.identify_structure()
                self.identify_fvg()
                self.execute_trades()
                self.calculate_performance()

                time.sleep(scan_interval)

        except KeyboardInterrupt:
            # Your existing interrupt handling...
            if self.in_trade:
                self.api.close_all_positions()
                self.in_trade = False

In [ ]:


def BitMEXLiveTrader():
    """
    Main function to run the BitMEXLiveTrader
    """
    # BitMEX API credentials (use your test API key and secret)
    #API_KEY = oAPI_KEY")  # Your test API key
    #API_SECRET = os.getenv("API_SECRET")  # Your test API secret


    try:
        # Call the custom logging configuration function
        #logger = configure_logging()
        print("Current time in SAST:", sast_now.strftime('%Y-%m-%d %H:%M:%S'))
        # Example log calls
        logger.info("This is BitmexSMCLiveTrader version 0.1 ")
        logger.error(f"Current time in SAST:{sast_now.strftime('%Y-%m-%d %H:%M:%S')}")
        # Initialize and run BitMEXLiveTrader
        trader = SMC(
            api_key=API_KEY,
            api_secret=API_SECRET,
            test=True,  # Use testnet
            symbol="SOLUSD",  # Solana/USD
            timeframe="15m",  # 5m candles
            risk_per_trade=0.02  # 2% risk per trade
        )

        # Start trading loop
        trader.run(scan_interval=120)  # Scan every 2 minutes

    except KeyboardInterrupt:
        logger.info("BitMEXLiveTrader stopped by user")
        print("BitMEXLiveTrader stopped by user")
    except Exception as e:
        logger.error(f"Unexpected error: {str(e)}")
        print(f"Unexpected error: {str(e)}")

In [ ]:
BitMEXLiveTrader()

2025-03-25 01:50:08,241 - INFO - This is BitmexSMCLiveTrader version 0.1 
INFO:__main__:This is BitmexSMCLiveTrader version 0.1 
2025-03-25 01:50:08,245 - ERROR - Current time in SAST:2025-03-25 03:50:08
ERROR:__main__:Current time in SAST:2025-03-25 03:50:08


Current time in SAST: 2025-03-25 03:50:08


/usr/local/lib/python3.11/dist-packages/swagger_spec_validator/validator20.py:49: SwaggerValidationWarning: Found "$ref: #/definitions/UserPreferences" with siblings that will be overwritten. See https://stackoverflow.com/a/48114924 for more information. (path #/definitions/User/properties/preferences)
  warnings.warn(
2025-03-25 01:50:09,978 - INFO - BitMEXTestAPI initialized for SOL-USD on testnet
INFO:__main__:BitMEXTestAPI initialized for SOL-USD on testnet
2025-03-25 01:50:09,983 - INFO - BitMEXLiveTrader initialized for SOLUSD on 5m timeframe
INFO:__main__:BitMEXLiveTrader initialized for SOLUSD on 5m timeframe
2025-03-25 01:50:09,986 - INFO - Starting BitMEXLiveTrader at 2025-03-25 03:50:09
INFO:__main__:Starting BitMEXLiveTrader at 2025-03-25 03:50:09
/usr/local/lib/python3.11/dist-packages/bravado_core/spec.py:469: Warning: JSON format is not registered with bravado-core!
  warnings.warn(
2025-03-25 01:50:10,171 - INFO - Profile information retrieved successfully
INFO:__main__

BitMEXTestAPI initialized for SOL-USD on testnet
Starting BitMEXLiveTrader at 2025-03-25 03:50:09
Profile information retrieved successfully
Account: mmehgoss+bitmexdemo@gmail.com
Wallet Balance: 0.01000000 XBT
Available Margin: 0.01000000 XBT
No open positions
Fetching SOLUSD market data from BitMEX
Failed to get data from BitMEX API: BitMEXTestAPI.get_candle() got an unexpected keyword argument 'symbol'. Falling back to yfinance.


[*********************100%***********************]  1 of 1 completed
2025-03-25 01:50:10,477 - INFO - Identifying market structure
INFO:__main__:Identifying market structure


...

2025-03-25 01:51:19,299 - INFO - Bearish FVG at index 35 has been mitigated at index 104
INFO:__main__:Bearish FVG at index 35 has been mitigated at index 104
2025-03-25 01:51:19,309 - INFO - Bullish FVG at index 40 has been mitigated at index 50
INFO:__main__:Bullish FVG at index 40 has been mitigated at index 50
2025-03-25 01:51:19,313 - INFO - Bearish FVG at index 44 has been mitigated at index 46
INFO:__main__:Bearish FVG at index 44 has been mitigated at index 46
2025-03-25 01:51:19,320 - INFO - Bearish FVG at index 45 has been mitigated at index 46
INFO:__main__:Bearish FVG at index 45 has been mitigated at index 46
2025-03-25 01:51:19,325 - INFO - Bearish FVG at index 48 has been mitigated at index 60
INFO:__main__:Bearish FVG at index 48 has been mitigated at index 60
2025-03-25 01:51:19,333 - INFO - Bearish FVG at index 49 has been mitigated at index 57
INFO:__main__:Bearish FVG at index 49 has been mitigated at index 57
2025-03-25 01:51:19,336 - INFO - Bearish FVG at index 50

Bearish FVG at index 35 has been mitigated at index 104
Bullish FVG at index 40 has been mitigated at index 50
Bearish FVG at index 44 has been mitigated at index 46
Bearish FVG at index 45 has been mitigated at index 46
Bearish FVG at index 48 has been mitigated at index 60
Bearish FVG at index 49 has been mitigated at index 57
Bearish FVG at index 50 has been mitigated at index 52
Bearish FVG at index 51 has been mitigated at index 52
Bearish FVG at index 54 has been mitigated at index 87
Bearish FVG at index 61 has been mitigated at index 62
Bearish FVG at index 66 has been mitigated at index 77
Bearish FVG at index 67 has been mitigated at index 75
Bearish FVG at index 68 has been mitigated at index 70
Bearish FVG at index 69 has been mitigated at index 70
Bullish FVG at index 70 has been mitigated at index 71
Bearish FVG at index 72 has been mitigated at index 73
Bearish FVG at index 80 has been mitigated at index 81
Bearish FVG at index 85 has been mitigated at index 123
Bearish 

INFO:__main__:Bullish FVG at index 113 has been mitigated at index 115
2025-03-25 01:51:19,512 - INFO - Bearish FVG at index 116 has been mitigated at index 118
INFO:__main__:Bearish FVG at index 116 has been mitigated at index 118
2025-03-25 01:51:19,519 - INFO - Bearish FVG at index 117 has been mitigated at index 118
INFO:__main__:Bearish FVG at index 117 has been mitigated at index 118
2025-03-25 01:51:19,530 - INFO - Bearish FVG at index 127 has been mitigated at index 133
INFO:__main__:Bearish FVG at index 127 has been mitigated at index 133
2025-03-25 01:51:19,533 - INFO - Bearish FVG at index 128 has been mitigated at index 129
INFO:__main__:Bearish FVG at index 128 has been mitigated at index 129
2025-03-25 01:51:19,545 - INFO - Bearish FVG at index 144 has been mitigated at index 148
INFO:__main__:Bearish FVG at index 144 has been mitigated at index 148
2025-03-25 01:51:19,554 - INFO - Bearish FVG at index 145 has been mitigated at index 148
INFO:__main__:Bearish FVG at index

Bullish FVG at index 113 has been mitigated at index 115
Bearish FVG at index 116 has been mitigated at index 118
Bearish FVG at index 117 has been mitigated at index 118
Bearish FVG at index 127 has been mitigated at index 133
Bearish FVG at index 128 has been mitigated at index 129
Bearish FVG at index 144 has been mitigated at index 148
Bearish FVG at index 145 has been mitigated at index 148
Bearish FVG at index 148 has been mitigated at index 150
Bearish FVG at index 149 has been mitigated at index 150
Bearish FVG at index 153 has been mitigated at index 155
Bearish FVG at index 154 has been mitigated at index 155
Bearish FVG at index 168 has been mitigated at index 169
Bearish FVG at index 170 has been mitigated at index 175
Bearish FVG at index 176 has been mitigated at index 177
Bearish FVG at index 177 has been mitigated at index 178
Bullish FVG at index 178 has been mitigated at index 180
Bearish FVG at index 187 has been mitigated at index 189
Bearish FVG at index 188 has be

2025-03-25 01:51:19,807 - INFO - Bullish FVG at index 22 has been mitigated at index 24
INFO:__main__:Bullish FVG at index 22 has been mitigated at index 24
2025-03-25 01:51:19,817 - INFO - Bullish FVG at index 23 has been mitigated at index 24
INFO:__main__:Bullish FVG at index 23 has been mitigated at index 24
2025-03-25 01:51:19,826 - INFO - Bearish FVG at index 25 has been mitigated at index 26
INFO:__main__:Bearish FVG at index 25 has been mitigated at index 26
2025-03-25 01:51:19,833 - INFO - Bearish FVG at index 28 has been mitigated at index 29
INFO:__main__:Bearish FVG at index 28 has been mitigated at index 29
2025-03-25 01:51:19,838 - INFO - Bullish FVG at index 30 has been mitigated at index 33
INFO:__main__:Bullish FVG at index 30 has been mitigated at index 33
2025-03-25 01:51:19,841 - INFO - Bullish FVG at index 32 has been mitigated at index 33
INFO:__main__:Bullish FVG at index 32 has been mitigated at index 33
2025-03-25 01:51:19,849 - INFO - Bearish FVG at index 34 h